<div style="font-size:18pt; padding-top:20px; text-align:center"><b>Классификация текстовых документов в </b> <span style="font-weight:bold; color:green">Spark</span></div><hr>
<div style="text-align:right;">Папулин С.Ю. <span style="font-style: italic;font-weight: bold;">(papulin_bmstu@mail.ru)</span></div>

<p>[ОПЦИОНАЛЬНО] <b>Настройка среды</b></p>

In [ ]:
import os
import sys

os.environ["SPARK_HOME"]="/opt/cloudera/parcels/SPARK2/lib/spark2"
os.environ["PYSPARK_PYTHON"]="/opt/rh/rh-python36/root/usr/bin/python"
os.environ["PYSPARK_DRIVER_PYTHON"]="/opt/rh/rh-python36/root/usr/bin/python"

spark_home = os.environ.get("SPARK_HOME")
sys.path.insert(0, os.path.join(spark_home, "python"))
sys.path.insert(0, os.path.join(spark_home, "python/lib/py4j-0.10.7-src.zip"))

<p>Запуск Spark Context</p>

In [ ]:
import pyspark

In [ ]:
conf = pyspark.SparkConf() \
        .setAppName("spamRDDApp") \
        .setMaster("yarn") \
        .set("spark.submit.deployMode", "client")

In [ ]:
sc = pyspark.SparkContext(conf=conf)

<p>Загрузите исходные данные из файла</p>

<p>[ЕСЛИ НЕОБХОДИМО] Скопируйте локальный датасет в HDFS</p>

In [ ]:
!hdfs dfs -mkdir -p data/spark_mllib_rdd

In [ ]:
!hdfs dfs -copyFromLocal data/spam-classification-data/SMSSpamCollection data/spark_mllib_rdd

In [ ]:
!hdfs dfs -ls data/spark_mllib_rdd

In [ ]:
file_path = "data/spark_mllib_rdd/SMSSpamCollection"

In [ ]:
rdd = sc.textFile(file_path)
rdd.count(), rdd.take(5)

<p>Преобразуйте исходный данные с текстом сообщений в коллекция LabeledPoint, где в качестве признаков выступает массив наличия вхождения по каждому индексу (соответствует терму - HashingTF преобразование)</p>

<a href="https://spark.apache.org/docs/2.2.0/api/python/pyspark.mllib.html#pyspark.mllib.feature.HashingTF">HashingTF</a>

In [ ]:
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.regression import LabeledPoint

In [ ]:
hashingTF = HashingTF(numFeatures=10000)
hashingTF.setBinary(True)

def parseText(row):
    arr = row.split("\t")
    text = map(lambda x: x.lower().strip(), arr[1].split())
    text_tf = hashingTF.transform(text)
    cl = 0 if arr[0] == "ham" else 1
    return LabeledPoint(cl, text_tf)

rdd_sms = rdd.map(parseText)
rdd_sms.take(5)

<p>Сформируйте обучающее и тестовое подмножества</p>

In [ ]:
train, test = rdd_sms.randomSplit([0.7, 0.3])

train.persist()
test.persist()

<p>Вычислите baseline для классификации</p>

In [ ]:
nonspam_test_count = test.filter(lambda x: x.label==0).count()
test_count = test.count()

nonspam_test_count, test_count

In [ ]:
accuracy_baseline = nonspam_test_count / float(test_count)
accuracy_baseline

<p>Произведите обучение модели наивного байесовского классификатора</p>

<a href="https://spark.apache.org/docs/2.2.0/api/python/pyspark.mllib.html#pyspark.mllib.classification.NaiveBayes">NaiveBayes</a>

In [ ]:
from pyspark.mllib.classification import NaiveBayes, NaiveBayesModel

In [ ]:
model_NB = NaiveBayes.train(train, 1.0)
model_NB

<p>Проверьте на тестовом подмножестве</p>

In [ ]:
pred_true_val = test.map(lambda row: (float(model_NB.predict(row.features)), row.label))
pred_true_val.count()
pred_true_val.take(5)

In [ ]:
test_count = test.count()
test_count

In [ ]:
nonspam_test_NB_count = pred_true_val.filter(lambda row: row[0]==row[1]).count()
nonspam_test_NB_count

In [ ]:
accuracy_NB = nonspam_test_NB_count / float(test_count)
accuracy_NB

<p>Другие метрики</p>

In [ ]:
from pyspark.mllib.evaluation import BinaryClassificationMetrics

In [ ]:
metrics = BinaryClassificationMetrics(pred_true_val)

In [ ]:
metrics.areaUnderROC

In [ ]:
metrics.areaUnderPR

<p>Завершение Spark контекста</p>

In [ ]:
sc.stop()

<a name="2"></a>
<div style="display:table; width:100%; padding-top:10px; padding-bottom:10px; border-bottom:1px solid lightgrey">
    <div style="display:table-row">
        <div style="display:table-cell; width:80%; font-size:14pt; font-weight:bold">2. Источники</div>
    	<div style="display:table-cell; width:20%; text-align:center; background-color:whitesmoke; border:1px solid lightgrey"><a href="#0">К содержанию</a></div>
    </div>
</div>

<a href="https://spark.apache.org/docs/2.2.0/mllib-guide.html">MLlib: RDD-based API</a>